### Open the NETCONF session

In [12]:
from ncclient import manager
vmx1=manager.connect(host="100.123.1.0", port="830", timeout=30, username="jcluser", password="Juniper!1", hostkey_verify=False)

In [14]:
vmx1.connected

True

### NETCONF capabilities

Print the NETCONF server capabilities

In [3]:
for item in vmx1.server_capabilities:
  print (item)

urn:ietf:params:netconf:base:1.0
urn:ietf:params:netconf:capability:candidate:1.0
urn:ietf:params:netconf:capability:confirmed-commit:1.0
urn:ietf:params:netconf:capability:validate:1.0
urn:ietf:params:netconf:capability:url:1.0?scheme=http,ftp,file
urn:ietf:params:xml:ns:netconf:base:1.0
urn:ietf:params:xml:ns:netconf:capability:candidate:1.0
urn:ietf:params:xml:ns:netconf:capability:confirmed-commit:1.0
urn:ietf:params:xml:ns:netconf:capability:validate:1.0
urn:ietf:params:xml:ns:netconf:capability:url:1.0?protocol=http,ftp,file
urn:ietf:params:xml:ns:yang:ietf-netconf-monitoring
http://xml.juniper.net/netconf/junos/1.0
http://xml.juniper.net/dmi/system/1.0


In [4]:
assert(":validate" in vmx1.server_capabilities), "NetConf server did not advertise the capability :validate"
assert(":candidate" in vmx1.server_capabilities), "NetConf server did not advertise the capability :candidate"
assert(":confirmed-commit" in vmx1.server_capabilities), "NetConf server did not advertise the capability :confirmed-commit"
assert(":writable-running" not in vmx1.server_capabilities), "NetConf server advertised the capability :writable-running"


Print the NETCONF client capabilities

In [5]:
for item in vmx1.client_capabilities:
  print (item)  

urn:ietf:params:netconf:base:1.0
urn:ietf:params:netconf:base:1.1
urn:ietf:params:netconf:capability:writable-running:1.0
urn:ietf:params:netconf:capability:candidate:1.0
urn:ietf:params:netconf:capability:confirmed-commit:1.0
urn:ietf:params:netconf:capability:rollback-on-error:1.0
urn:ietf:params:netconf:capability:startup:1.0
urn:ietf:params:netconf:capability:url:1.0?scheme=http,ftp,file,https,sftp
urn:ietf:params:netconf:capability:validate:1.0
urn:ietf:params:netconf:capability:xpath:1.0
urn:ietf:params:netconf:capability:notification:1.0
urn:liberouter:params:netconf:capability:power-control:1.0
urn:ietf:params:netconf:capability:interleave:1.0
urn:ietf:params:netconf:capability:with-defaults:1.0


### Retrieve the configuration using the `get_config` NETCONF operation

Backup the active configuration on the host running this script

In [6]:
f=open ("vmx1_config.xml", 'w')
f.write(str(vmx1.get_config(source='running')))
f.close()

Print the candidate configuration

In [7]:
# print (vmx1.get_config(source="candidate"))

Use a subtree filter to get only interfaces configuration from the active configuration. These 2 examples provide the same output

In [8]:
criteria1='''
<configuration>
     <interfaces>
     </interfaces>
</configuration>
'''
interfaces_config_1 = vmx1.get_config(source="running", filter=("subtree", criteria1))
# print (interfaces_config_1)


criteria2='''
<configuration>
     <interfaces/>
</configuration>
'''
interfaces_config_2 = vmx1.get_config(source="running", filter=("subtree", criteria2))
# print (interfaces_config_2)


### Update the candidate configuration and commit

Use the NETCONF operations `lock`, `edit_config` and `unlock` to update the configuration

In [33]:
criteria='''
<configuration>
   <system>
     <host-name/>
   </system>
</configuration>
'''
vmx1.get_config(source="running", filter=("subtree", criteria))

<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:junos="http://xml.juniper.net/junos/18.2R1/junos" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:7bb93654-cdd3-4d35-9509-2608bba2288c">
<data>
<configuration xmlns="http://xml.juniper.net/xnm/1.1/xnm" junos:commit-seconds="1574863805" junos:commit-localtime="2019-11-27 14:10:05 UTC" junos:commit-user="jcluser">
    <system>
        <host-name>spine1</host-name>
    </system>
</configuration>
</data>
</rpc-reply>

In [34]:
conf='''<config><configuration><system><host-name operation="replace">vmx1</host-name></system></configuration></config>'''
vmx1.lock('candidate')
vmx1.edit_config(target='candidate', config=conf)
vmx1.commit()
vmx1.unlock('candidate')


<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:junos="http://xml.juniper.net/junos/18.2R1/junos" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:9e8ed585-3fb3-4d01-9ba6-e03100f2d541">
<ok/>
</rpc-reply>

In [35]:
criteria='''
<configuration>
   <system>
     <host-name/>
   </system>
</configuration>
'''
vmx1.get_config(source="running", filter=("subtree", criteria))

<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:junos="http://xml.juniper.net/junos/18.2R1/junos" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:61984423-5fe5-4093-a15c-a649e5aa1179">
<data>
<configuration xmlns="http://xml.juniper.net/xnm/1.1/xnm" junos:commit-seconds="1574863822" junos:commit-localtime="2019-11-27 14:10:22 UTC" junos:commit-user="jcluser">
    <system>
        <host-name>vmx1</host-name>
    </system>
</configuration>
</data>
</rpc-reply>

### Revert a candidate configuration to the currently running configuration

Update the candidate configuration

In [37]:
conf = '''<config><configuration><system><host-name operation="replace">anothername</host-name></system></configuration></config>'''
vmx1.edit_config(target='candidate', config=conf)

<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:junos="http://xml.juniper.net/junos/18.2R1/junos" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:2177586f-9dd6-42ca-92a0-2533ecac1d7a">
<ok/>
</rpc-reply>

Get the candidate configuration

In [40]:
criteria='''
<configuration>
   <system>
     <host-name/>
   </system>
</configuration>
'''
vmx1.get_config(source="candidate", filter=("subtree", criteria))

<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:junos="http://xml.juniper.net/junos/18.2R1/junos" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:91932dcf-c696-4a46-ba75-33c3ad049b2a">
<data>
<configuration xmlns="http://xml.juniper.net/xnm/1.1/xnm" junos:changed-seconds="1574864008" junos:changed-localtime="2019-11-27 14:13:28 UTC">
    <system>
        <host-name>anothername</host-name>
    </system>
</configuration>
</data>
</rpc-reply>

Revert the candidate configuration to the currently running configuration. Any uncommitted changes are discarded.

In [41]:
vmx1.discard_changes()

<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:junos="http://xml.juniper.net/junos/18.2R1/junos" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:3d5c5b88-da65-4876-8a1c-f6a5015cf550">
<ok/>
</rpc-reply>

Check the candidate configuration

In [42]:
vmx1.get_config(source="candidate", filter=("subtree", criteria))

<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:junos="http://xml.juniper.net/junos/18.2R1/junos" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:dcaeabb8-5659-4d7c-b713-965a1d287379">
<data>
<configuration xmlns="http://xml.juniper.net/xnm/1.1/xnm" junos:changed-seconds="1574864215" junos:changed-localtime="2019-11-27 14:16:55 UTC">
    <system>
        <host-name>vmx1</host-name>
    </system>
</configuration>
</data>
</rpc-reply>

### Close the NETCONF session

Use the `close_session` NETCONF operation

In [10]:
vmx1.close_session()

<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:junos="http://xml.juniper.net/junos/18.2R1/junos" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:1b3744d1-f6ac-4045-9a1a-4a4e2386265b">
<ok/>
</rpc-reply>

In [11]:
vmx1.connected

False